## Imports

In [1]:
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os
import warnings

warnings.simplefilter("ignore", category=UserWarning)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Environment

In [8]:
import ale_py

env = gym.make("ALE/Blackjack-v5", render_mode="human")

In [9]:
env.reset()

(array([[[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
         [110, 156,  66],
         [110, 156,  66]],
 
        [[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
         [110, 156,  66],
         [110, 156,  66]],
 
        [[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
         [110, 156,  66],
         [110, 156,  66]],
 
        ...,
 
        [[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
         [110, 156,  66],
         [110, 156,  66]],
 
        [[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
         [110, 156,  66],
         [110, 156,  66]],
 
        [[110, 156,  66],
         [110, 156,  66],
         [110, 156,  66],
         ...,
         [110, 156,  66],
  

In [10]:
env.action_space

Discrete(4)

In [11]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [12]:
env.render()

## Train Model

In [17]:
env = DummyVecEnv([lambda: env])

In [18]:
log_path = os.path.join("Training", "logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log = log_path, device=device)

Using mps device
Wrapping the env in a VecTransposeImage.


In [19]:
model.learn(total_timesteps=100000)

Logging to Training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 21   |
|    iterations      | 1    |
|    time_elapsed    | 94   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 22          |
|    iterations           | 2           |
|    time_elapsed         | 183         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017205196 |
|    clip_fraction        | 0.0983      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | 0.000469    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.3         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00523    |
|    value_loss           | 4.5         |
-----------------------------------------
---

## Save Model

In [20]:
ppo_path = os.path.join("Training", "Saved Models", "PPO_Driving_Model")

In [21]:
model.save(ppo_path)

In [22]:
del model

In [23]:
model = PPO.load(ppo_path, env)

Wrapping the env in a VecTransposeImage.


## Test Model

In [24]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(0.0, 0.0)

In [25]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: [-68.]
Episode: 2 Score: [-34.]
Episode: 3 Score: [14.]
Episode: 4 Score: [-56.]
Episode: 5 Score: [-8.]
